# Building a Digital Orrery

----

By Adam A Miller (Northwestern/CIERA/SkAI)  
09 Sept 2025

**Version 0.1**

(based on a problem originally written by Jeff Oishi [for the DSFP](https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions/blob/main/Sessions/Session08/Day1/OOP_problem.ipynb))

In this problem you will construct a Digital Orrery. An [orrery](https://en.wikipedia.org/wiki/Orrery) is a mechanical model of the Solar System. Here, we will generalize this to anything that is mechanically similar to *the* solar system: a collection of things bound gravitationally. 

<img src="https://upload.wikimedia.org/wikipedia/commons/4/48/Grand_orrery_in_Putnam_Gallery%2C_2009-11-24.jpg" alt="Orrery" width="600"/>
(image: wikimedia)


In [1]:
import numpy as np
# import matplotlib.pyplot as plt

## Problem 1) Building a basic set of objects

Our first task is to map our problem onto a set of **objects** that we **instantiate** (that is, make **instances** of) in order to solve our problem.

Let's outline the scope of our problem.

A solar system exists in a Universe; here we can ignore the gravitational perturbation on the Solar System from the rest of the Universe (i.e., the Orrery can be treated as the only thing in the Universe). Our model will consist of a small number of bodies containing mass. It might also contain bodies without mass, so called "test particles."

The problem to be solved numerically is the gravitational N-body problem,

$$\ddot{\mathbf{r}}_i = -G\sum_{i \ne j} \frac{m_j \mathbf{r}_{ij}}{r_{ij}^3},$$

where $\mathbf{r}_{ij} \equiv \mathbf{r_i} - \mathbf{r_j}$. This task itself can be broken into two components: 

* the force calculation
* the ODE integrator to advance $\mathbf{r}_i$ and $\dot{\mathbf{r}}_i$ forward in time

**Problem 1a**

In disucssion with a classmate, sketch out a set of classes that you will need to complete this project. Don't worry about things like numerical integrators yet. 

Also, sketch out interfaces (start with the class constructor), but don't worry about writing code right now.

*Hint* – what is the minimal number of classes you will need to perform the calculation?

*write your response here*

To build the Orrery we will need two classes. One called `Body`, which can be used to hold instances of the different objects that are gravitationally interacting. The important properties for a body are its mass, position, velocity, and acceleration. 

We also need a `Universe` class, which will include all the bodies that are present within the Universe. 

In [2]:
class Body():
    pass

class Universe():
    pass

**Problem 1b**

Using `r0` and `rdot0` below create an instance of the `Body` class. 

*Hint* – if you did not fully create the `Body` class in the previous problem do that here.

In [3]:
class Body: 
    def __init__(self, m, r, rdot): 
        self.m = m 
        self.r = np.array(r, dtype=float) 
        self.rdot = np.array(rdot, dtype=float) 
        self.a = np.zeros_like(self.r)

In [4]:
r0 = np.array([0,0,0])
rdot0 = np.array([0,0,0])

In [5]:
b = Body(1,r0, rdot0)

## Problem 2

Now, we code the numerical algorithms. We're going to do the most simple things possible: a *brute force* ("direct N-Body" if you're feeling fancy) force calculation, and a leapfrog time integrator.

The leapfrog scheme is an explicit, second order scheme given by

$$r_{i+1} = r_{i} + v_{i} \Delta t + \frac{\Delta t^2}{2} a_{i}$$

$$v_{i+1} = v_{i} + \frac{\Delta t}{2} (a_{i} + a_{i+1}),$$

where $\Delta t$ is the time step (which we'll just keep constant), and the subscript refers to the *iteration* number $i$. 

Note that this scheme requires a force update *in between* calculating $r_{i+1}$ and $v_{i+1}$. (In other words, the code should proceed a half time step to update the velocity of the bodies, then calculate the new position of the bodies using a full time step, compute the forces, and then proceed with an additional half time step to get the updated velocity.)

**Problem 2a** 

Write a method that implements the force integrator. Test it on simple cases:
 * two equal 1 $M_\odot$ objects in your universe, 1 AU apart
 * a $1\ M_\odot$ object and a $1\ M_{\oplus}$ object, 1 AU apart

In [8]:
class Universe: 
    def __init__(self): 
        self.bodies = [] 
    
    def add_body(self, body): 
        self.bodies.append(body)

    def compute_forces(self): 
        G = 6.67430e-11 # gravitational constant 
        for body in self.bodies: 
            body.a = np.zeros_like(body.r) 
        for i, bi in enumerate(self.bodies): 
            for j, bj in enumerate(self.bodies): 
                if i != j: 
                    rij = bj.r - bi.r 
                    dist = sum(rij**2)**0.5 
                    if dist > 0: 
                        bi.a += G * bj.m * rij / dist**3

In [9]:
# test case 1 two equal mass bodies separated by 1 AU

body1 = Body(1.0, [0, 0, 0], [0, 0, 0])
body2 = Body(1.0, [1, 0, 0], [0, 0, 0])
em_uni = Universe()
em_uni.add_body(body1)
em_uni.add_body(body2)
em_uni.compute_forces()
print("Body 1:", body1.a)
print("Body 2:", body2.a)


Body 1: [6.6743e-11 0.0000e+00 0.0000e+00]
Body 2: [-6.6743e-11  0.0000e+00  0.0000e+00]


In [10]:
# test case 2 - sun + earth system separated by 1 AU

body_sun = Body(1.0, [0, 0, 0], [0, 0, 0])
body_earth = Body(3.003e-6, [1, 0, 0], [0, 0, 0])
se_uni = Universe()
se_uni.add_body(body_sun)
se_uni.add_body(body_earth)
se_uni.compute_forces()
print("Sun:", body_sun.a)
print("Earth:", body_earth.a)

Sun: [2.00429229e-16 0.00000000e+00 0.00000000e+00]
Earth: [-6.6743e-11  0.0000e+00  0.0000e+00]


**Problem 2b**
Write the leapfrog integration as a method in the `Universe` class. Test it on one particle with no force (what should it do?)

In [21]:
class Universe: 
    def __init__(self): 
        self.bodies = [] 
    
    def add_body(self, body): 
        self.bodies.append(body)

    def compute_forces(self): 
        G = 6.67430e-11 # gravitational constant 
        for body in self.bodies: 
            body.a = np.zeros_like(body.r) 
        for i, bi in enumerate(self.bodies): 
            for j, bj in enumerate(self.bodies): 
                if i != j: 
                    rij = bj.r - bi.r 
                    dist = sum(rij**2)**0.5 
                    if dist > 0: 
                        bi.a += G * bj.m * rij / dist**3

    def leapfrog_step(self, dt): 
        # First half-step for velocity 
        for body in self.bodies: 
            body.rdot += 0.5 * dt * body.a 
        # Full step for position 
        for body in self.bodies: 
            body.r += dt * body.rdot
        
        # Update forces 
        self.compute_forces() 
        
        # Second half-step for velocity 
        for body in self.bodies: 
            body.rdot += 0.5 * dt * body.a

In [22]:
# Test case: One particle with no force
body = Body(1.0, [0, 0, 0], [1, 0, 0])
no_force = Universe()
no_force.add_body(body)

for pos_num, pos in enumerate(range(10)):
    no_force.leapfrog_step(60)
    for b in no_force.bodies:
        print(f'At timestep {pos_num}, the position is {b.r}')




At timestep 0, the position is [60.  0.  0.]
At timestep 1, the position is [120.   0.   0.]
At timestep 2, the position is [180.   0.   0.]
At timestep 3, the position is [240.   0.   0.]
At timestep 4, the position is [300.   0.   0.]
At timestep 5, the position is [360.   0.   0.]
At timestep 6, the position is [420.   0.   0.]
At timestep 7, the position is [480.   0.   0.]
At timestep 8, the position is [540.   0.   0.]
At timestep 9, the position is [600.   0.   0.]


**Problem 2c** 
 
Wire it all up! Try a 3-body calculation of the Earth-Sun-Moon system. Try the Earth-Jupiter-Sun system! 

In [32]:
# Masses in kg (Note - this was set by the choice of G earlier)
M_sun = 1.989e30 
M_earth = 5.972e24 
M_moon = 7.348e22 

# Distances in meters 
AU = 1.496e11 
moon_dist = 3.844e8 

# Initial positions and velocities 
sun = Body(M_sun, [0, 0, 0], [0, 0, 0]) 
earth = Body(M_earth, [AU, 0, 0], [0, 29780, 0]) # 29.78 km/s 
moon = Body(M_moon, [AU + moon_dist, 0, 0], [0, 29780 + 1022, 0]) # Moon velocity 

u = Universe() 
u.add_body(sun) 
u.add_body(earth) 
u.add_body(moon) 

u.compute_forces() 

n_steps = 1000
for step_num in range(n_steps): 
    u.leapfrog_step(3600) # 1 hour timestep

print(f'After {n_steps} hours, the bodies have now moved to:')
for b in u.bodies:
    print(f'The position of this body is {b.r}')

After 1000 hours, the bodies have now moved to:
The position of this body is [111900.43764608  27206.52992804      0.        ]
The position of this body is [1.12793032e+11 9.83030251e+10 0.00000000e+00]
The position of this body is [1.12440066e+11 9.81860844e+10 0.00000000e+00]


Note a few things that "make sense" in this instance. 

1. The sun has "barely" moved
2. None of the bodies have any motion outside the orbital plane.
3. The moon is being "dragged" by the Earth (i.e., the moon is clearly staying with the Earth)

What about 1 year - if the system is advanced by $24 \times 365.25 \approx 8768\,\mathrm{hr}$ - do the bodies return to their original position?

In [33]:
# Initial positions and velocities 
sun = Body(M_sun, [0, 0, 0], [0, 0, 0]) 
earth = Body(M_earth, [AU, 0, 0], [0, 29780, 0]) # 29.78 km/s 
moon = Body(M_moon, [AU + moon_dist, 0, 0], [0, 29780 + 1022, 0]) # Moon velocity 

u1 = Universe() 
u1.add_body(sun) 
u1.add_body(earth) 
u1.add_body(moon) 

u1.compute_forces() 

n_steps = 8768
earth_pos = []
for step_num in range(n_steps): 
    u1.leapfrog_step(3600) # 1 hour timestep


print(f'After {n_steps} hours, the bodies have now moved to:')
for b in u1.bodies:
    print(f'The position of this body is {b.r/AU}')

After 8768 hours, the bodies have now moved to:
The position of this body is [6.79131885e-13 1.91081260e-05 0.00000000e+00]
The position of this body is [ 1.00005470e+00 -6.30019076e-04  0.00000000e+00]
The position of this body is [ 0.99810544 -0.00218689  0.        ]


## Challenge Problem

* Construct a visualization method for the `Universe` class
* Read about the Fast Multipole Method (FMM) [here](https://math.nyu.edu/faculty/greengar/shortcourse_fmm.pdf) and implement one for the force calculation

In [ ]:
# good luck!